In [ ]:
pip install --upgrade gensim

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 24.1 MB 1.5 MB/s 
  Attempting uninstall: gensim
    Found existing installation: gensim 3.6.0
    Uninstalling gensim-3.6.0:
      Successfully uninstalled gensim-3.6.0


In [ ]:
import pandas as pd
import numpy as np
import ast
import re
import sklearn
import six
from abc import ABCMeta
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.utils.extmath import safe_sparse_dot
from scipy import sparse
import warnings
warnings.filterwarnings('ignore')
import nltk
import six
import re
import gensim
from tqdm import tqdm


from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.svm import LinearSVC
from abc import ABCMeta
from sklearn.pipeline import Pipeline
from sklearn.utils import check_X_y, check_array
from scipy import sparse
from nltk.stem import PorterStemmer,WordNetLemmatizer
from scipy.sparse import issparse
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.utils.extmath import safe_sparse_dot
from sklearn.preprocessing import normalize, binarize, LabelBinarizer
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer,TfidfTransformer
from sklearn.utils import resample
from sklearn.metrics import accuracy_score,f1_score
from gensim.models import FastText

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
tsel = pd.read_csv(r'/content/drive/MyDrive/TA/Prepro_Pelayanan.csv', usecols=['Username','Text','Pelayanan','text_result'])
tsel

,Username,Text,Pelayanan,text_result
0,satriafakhri_id,@yamasaiyaa untuk masalah hp lapor aja ke yg b...,0.0,"['handphone', 'lapor', 'wajib', 'bantu', 'laca..."
1,Piyuuuuu21_,"@bertanyarl Gatau, akhir akhir ini Telkomsel l...",0.0,"['tidak tahu', 'telkomsel', 'lambat', 'kuota',..."
2,hypersex_pr,Ada kendala kah dengan sinyal telkomsel,0.0,"['kendala', 'sinyal', 'telkomsel']"
3,snwyscrtimh,@squidwardpemala @Cutimut3 @reihanahsan @sonde...,-1.0,"['tidak bisa', 'nomor', 'mati', 'dihidupakan',..."
4,aninadhany,"@Telkomsel min, dari kmrn aplikasi my telkomse...",-1.0,"['admin', 'kemarin', 'aplikasi', 'punya', 'tel..."
...,...,...,...,...
16986,Syaeeef,"Sinyal mines, #sinyal #telkomsel #byu.id tolon...",0.0,"['sinyal', 'minus', 'tolong', 'baik', 'problem..."
16987,abdr40,@Telkomsel ini My Telkomsel gangguan sampe kap...,0.0,"['punya', 'telkomsel', 'ganggu', 'aduh', 'desa..."
16988,thepinediary,yaallah nambah2i gawean waeee esuk2 kudu neng ...,0.0,"['allah', 'tambah', 'kerja', 'esuk', 'neng', '..."
16989,yudaamaraa,@Telkomsel min sinyal telkomsel di rumah saya ...,0.0,"['admin', 'sinyal', 'telkomsel', 'rumah', 'dae..."


In [ ]:
tsel['text_result'] = tsel['text_result'].apply(str)

In [ ]:
import ast

def join_text_list(text):
    texts = ast.literal_eval(text)
    return ' '.join([text for text in texts])

tsel['text_result'] = tsel['text_result'].apply(join_text_list)
tsel

,Username,Text,Pelayanan,text_result
0,satriafakhri_id,@yamasaiyaa untuk masalah hp lapor aja ke yg b...,0.0,handphone lapor wajib bantu lacak imei perihal...
1,Piyuuuuu21_,"@bertanyarl Gatau, akhir akhir ini Telkomsel l...",0.0,tidak tahu telkomsel lambat kuota promo murah ...
2,hypersex_pr,Ada kendala kah dengan sinyal telkomsel,0.0,kendala sinyal telkomsel
3,snwyscrtimh,@squidwardpemala @Cutimut3 @reihanahsan @sonde...,-1.0,tidak bisa nomor mati dihidupakan grapari nomo...
4,aninadhany,"@Telkomsel min, dari kmrn aplikasi my telkomse...",-1.0,admin kemarin aplikasi punya telkomsel ganggu ...
...,...,...,...,...
16986,Syaeeef,"Sinyal mines, #sinyal #telkomsel #byu.id tolon...",0.0,sinyal minus tolong baik problem location cama...
16987,abdr40,@Telkomsel ini My Telkomsel gangguan sampe kap...,0.0,punya telkomsel ganggu aduh desak pakai
16988,thepinediary,yaallah nambah2i gawean waeee esuk2 kudu neng ...,0.0,allah tambah kerja esuk neng grapari
16989,yudaamaraa,@Telkomsel min sinyal telkomsel di rumah saya ...,0.0,admin sinyal telkomsel rumah daerah rawabunga ...


## Ektraksi Fitur TF-IDF 

In [ ]:
vectors= TfidfVectorizer(ngram_range=(1,2), max_features = 1000)
vectors

TfidfVectorizer(max_features=1000, ngram_range=(1, 2))

In [ ]:
tfidf = vectors.fit_transform(tsel['text_result'].astype(str))
tfidf

<16991x1000 sparse matrix of type '<class 'numpy.float64'>'
	with 116944 stored elements in Compressed Sparse Row format>

In [ ]:
df_tfidf = tfidf.copy()

In [ ]:
X = df_tfidf
y = tsel['Pelayanan']

### SMOTE

In [ ]:
pip install imblearn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from imblearn.over_sampling import SMOTE
smote = SMOTE()

In [ ]:
X_smote, y_smote = smote.fit_resample(X,y)

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(X_smote,y_smote, test_size=0.1, random_state=45)

In [ ]:
y_smote.value_counts()

 0.0    13548
-1.0    13548
 1.0    13548
Name: Pelayanan, dtype: int64

## Ekspansi Fitur

In [ ]:
# Load Model trained
model_exp = FastText.load('/content/drive/MyDrive/TA/model_ntwt/fasttext/Twitter_Berita.fasttext').wv

# Penggunaan corpus disesuaikan dengan menggunakan corpus Twitter, Berita, Twitter+Berita

In [ ]:
def feature_expansion(df,feature):
    for col in tqdm(df.columns): #loop per kolom
        try: 
            sim_word = model_exp.similar_by_word(col, topn = 20) #Mencari similarity berdasarkan nilai n
        except:
            sim_word = []
        if sim_word != []: #jika similarity tidak kosong
            for term in [sim_word[i][0] for i in range(len(sim_word))]: #loop per-word yang ada di Similarity
                if term in feature:
                    #untuk semua kolom yang mempunyai nilai 0 di kolom, tetapi mempunyai nilai yang bukan 0 pada kolom term
                    #nilainya diganti dengan nilai kolom term yang mempunyai nilai bukan 0
                    df[col][(df[col]==0) & (df[term]!=0)] = df[term][(df[col]==0) & (df[term]!=0)]
    return df

In [ ]:
feature_ef = vectors.get_feature_names()

In [ ]:
# Membuat dataframe data test & train
tsel_x_train = pd.DataFrame(x_train.todense(), columns = feature_ef)
tsel_x_test = pd.DataFrame(x_test.todense(), columns = feature_ef)

In [ ]:
dfexp_x_train= feature_expansion(tsel_x_train, feature_ef)

100%|██████████| 1000/1000 [00:32<00:00, 30.49it/s]


In [ ]:
dfexp_x_test = feature_expansion(tsel_x_test, feature_ef)

100%|██████████| 1000/1000 [00:15<00:00, 65.00it/s]


## Klasifikasi

In [ ]:
class NBSVM(six.with_metaclass(ABCMeta, BaseEstimator, ClassifierMixin)):
    def __init__(self, alpha=1.0, C=1.0, max_iter=10000):
        self.alpha = alpha
        self.max_iter = max_iter
        self.C = C
        self.svm_ = [] # fuggly

    def fit(self, X, y):
        X, y = check_X_y(X, y, 'csr')
        _, n_features = X.shape

        labelbin = LabelBinarizer()
        Y = labelbin.fit_transform(y)
        self.classes_ = labelbin.classes_
        if Y.shape[1] == 1:
            Y = np.concatenate((1 - Y, Y), axis=1)

        # LabelBinarizer().fit_transform() returns arrays with dtype=np.int64.
        # so we don't have to cast X to floating point
        Y = Y.astype(np.float64)

        # Count raw events from data
        n_effective_classes = Y.shape[1]
        self.class_count_ = np.zeros(n_effective_classes, dtype=np.float64)
        self.ratios_ = np.full((n_effective_classes, n_features), self.alpha,
                                 dtype=np.float64)
        self._compute_ratios(X, Y)

        # flugglyness
        for i in range(n_effective_classes):
            X_i = X.multiply(self.ratios_[i])
            svm = LinearSVC(C=self.C, max_iter=self.max_iter)
            Y_i = Y[:,i]
            svm.fit(X_i, Y_i)
            self.svm_.append(svm) 

        return self

    def predict(self, X):
        n_effective_classes = self.class_count_.shape[0]
        n_examples = X.shape[0]

        D = np.zeros((n_effective_classes, n_examples))

        for i in range(n_effective_classes):
            X_i = X.multiply(self.ratios_[i])
            D[i] = self.svm_[i].decision_function(X_i)
        
        return self.classes_[np.argmax(D, axis=0)]
        
    def _compute_ratios(self, X, Y):
        """Count feature occurrences and compute ratios."""
        if np.any((X.data if issparse(X) else X) < 0):
            raise ValueError("Input X must be non-negative")

        self.ratios_ += safe_sparse_dot(Y.T, X)  # ratio + feature_occurrance_c
        normalize(self.ratios_, norm='l1', axis=1, copy=False)
        row_calc = lambda r: np.log(np.divide(r, (1 - r)))
        self.ratios_ = np.apply_along_axis(row_calc, axis=1, arr=self.ratios_)
        check_array(self.ratios_)
        self.ratios_ = sparse.csr_matrix(self.ratios_)

In [ ]:
x_train_csr = sparse.csr_matrix(dfexp_x_train)
x_test_csr = sparse.csr_matrix(dfexp_x_test)
model = NBSVM()

model.fit(x_train_csr,y_train)
y_pred_exp = model.predict(x_test_csr)
y_true = y_test

# Pengujian 5

### Top 1

In [ ]:
#example training
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
#[None, 'micro', 'macro', 'weighted'].
print(confusion_matrix(y_true,y_pred_exp))
print(classification_report(y_true,y_pred_exp))
print("Precision: ",precision_score(y_true, y_pred_exp, average="macro"))
print("Recall: ",recall_score(y_true, y_pred_exp, average="macro"))
print("F1: ",f1_score(y_true, y_pred_exp,  average="macro"))
print("Accuracy: ",accuracy_score(y_true, y_pred_exp))

[[1139   73  118]
 [ 114 1112  142]
 [ 113   20 1234]]
              precision    recall  f1-score   support

        -1.0       0.83      0.86      0.84      1330
         0.0       0.92      0.81      0.86      1368
         1.0       0.83      0.90      0.86      1367

    accuracy                           0.86      4065
   macro avg       0.86      0.86      0.86      4065
weighted avg       0.86      0.86      0.86      4065

Precision:  0.8608711673022378
Recall:  0.8573210438108602
F1:  0.8573172000825698
Accuracy:  0.8573185731857319


#### ROC-AUC

In [ ]:
from sklearn.preprocessing import OneHotEncoder

In [ ]:
enc = OneHotEncoder(handle_unknown='ignore')
y_pred_roc = enc.fit_transform(np.array(y_pred_exp).reshape(-1,1))
Y_pred = y_pred_roc.toarray()

In [ ]:
y_true_r = enc.fit_transform(np.array(y_true).reshape(-1,1))
y_true_roc = y_true_r.toarray()

In [ ]:
 from sklearn.metrics import roc_auc_score
 roc_auc_score(y_true_roc,Y_pred, multi_class='ovr')

0.8930190868671114

### Top 5

In [ ]:
#example training
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
#[None, 'micro', 'macro', 'weighted'].
print(confusion_matrix(y_true,y_pred_exp))
print(classification_report(y_true,y_pred_exp))
print("Precision: ",precision_score(y_true, y_pred_exp, average="macro"))
print("Recall: ",recall_score(y_true, y_pred_exp, average="macro"))
print("F1: ",f1_score(y_true, y_pred_exp,  average="macro"))
print("Accuracy: ",accuracy_score(y_true, y_pred_exp))

[[1139   73  118]
 [ 114 1112  142]
 [ 113   20 1234]]
              precision    recall  f1-score   support

        -1.0       0.83      0.86      0.84      1330
         0.0       0.92      0.81      0.86      1368
         1.0       0.83      0.90      0.86      1367

    accuracy                           0.86      4065
   macro avg       0.86      0.86      0.86      4065
weighted avg       0.86      0.86      0.86      4065

Precision:  0.8608711673022378
Recall:  0.8573210438108602
F1:  0.8573172000825698
Accuracy:  0.8573185731857319


#### ROC-AUC

In [ ]:
from sklearn.preprocessing import OneHotEncoder

In [ ]:
enc = OneHotEncoder(handle_unknown='ignore')
y_pred_roc = enc.fit_transform(np.array(y_pred_exp).reshape(-1,1))
Y_pred = y_pred_roc.toarray()

In [ ]:
y_true_r = enc.fit_transform(np.array(y_true).reshape(-1,1))
y_true_roc = y_true_r.toarray()

In [ ]:
 from sklearn.metrics import roc_auc_score
 roc_auc_score(y_true_roc,Y_pred, multi_class='ovr')

0.8930190868671114

### Top 10

In [ ]:
#example training
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
#[None, 'micro', 'macro', 'weighted'].
print(confusion_matrix(y_true,y_pred_exp))
print(classification_report(y_true,y_pred_exp))
print("Precision: ",precision_score(y_true, y_pred_exp, average="macro"))
print("Recall: ",recall_score(y_true, y_pred_exp, average="macro"))
print("F1: ",f1_score(y_true, y_pred_exp,  average="macro"))
print("Accuracy: ",accuracy_score(y_true, y_pred_exp))

[[1141   66  123]
 [ 120 1111  137]
 [ 120   24 1223]]
              precision    recall  f1-score   support

        -1.0       0.83      0.86      0.84      1330
         0.0       0.93      0.81      0.86      1368
         1.0       0.82      0.89      0.86      1367

    accuracy                           0.85      4065
   macro avg       0.86      0.85      0.85      4065
weighted avg       0.86      0.85      0.86      4065

Precision:  0.8586516801582879
Recall:  0.8548963596099083
F1:  0.8549764704144033
Accuracy:  0.8548585485854858


### Top 20

In [ ]:
#example training
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
#[None, 'micro', 'macro', 'weighted'].
print(confusion_matrix(y_true,y_pred_exp))
print(classification_report(y_true,y_pred_exp))
print("Precision: ",precision_score(y_true, y_pred_exp, average="macro"))
print("Recall: ",recall_score(y_true, y_pred_exp, average="macro"))
print("F1: ",f1_score(y_true, y_pred_exp,  average="macro"))
print("Accuracy: ",accuracy_score(y_true, y_pred_exp))

[[1128   66  136]
 [ 118 1114  136]
 [ 126   31 1210]]
              precision    recall  f1-score   support

        -1.0       0.82      0.85      0.83      1330
         0.0       0.92      0.81      0.86      1368
         1.0       0.82      0.89      0.85      1367

    accuracy                           0.85      4065
   macro avg       0.85      0.85      0.85      4065
weighted avg       0.85      0.85      0.85      4065

Precision:  0.852840860086472
Recall:  0.8491992498518507
F1:  0.8494195565940851
Accuracy:  0.8492004920049201


# Pengujian 5

### Top 1

In [ ]:
#example training
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
#[None, 'micro', 'macro', 'weighted'].
print(confusion_matrix(y_true,y_pred_exp))
print(classification_report(y_true,y_pred_exp))
print("Precision: ",precision_score(y_true, y_pred_exp, average="macro"))
print("Recall: ",recall_score(y_true, y_pred_exp, average="macro"))
print("F1: ",f1_score(y_true, y_pred_exp,  average="macro"))
print("Accuracy: ",accuracy_score(y_true, y_pred_exp))

[[1139   73  118]
 [ 114 1112  142]
 [ 113   20 1234]]
              precision    recall  f1-score   support

        -1.0       0.83      0.86      0.84      1330
         0.0       0.92      0.81      0.86      1368
         1.0       0.83      0.90      0.86      1367

    accuracy                           0.86      4065
   macro avg       0.86      0.86      0.86      4065
weighted avg       0.86      0.86      0.86      4065

Precision:  0.8608711673022378
Recall:  0.8573210438108602
F1:  0.8573172000825698
Accuracy:  0.8573185731857319


#### ROC-AUC

In [ ]:
from sklearn.preprocessing import OneHotEncoder

In [ ]:
enc = OneHotEncoder(handle_unknown='ignore')
y_pred_roc = enc.fit_transform(np.array(y_pred_exp).reshape(-1,1))
Y_pred = y_pred_roc.toarray()

In [ ]:
y_true_r = enc.fit_transform(np.array(y_true).reshape(-1,1))
y_true_roc = y_true_r.toarray()

In [ ]:
 from sklearn.metrics import roc_auc_score
 roc_auc_score(y_true_roc,Y_pred, multi_class='ovr')

0.8930190868671114

### Top 5

In [ ]:
#example training
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
#[None, 'micro', 'macro', 'weighted'].
print(confusion_matrix(y_true,y_pred_exp))
print(classification_report(y_true,y_pred_exp))
print("Precision: ",precision_score(y_true, y_pred_exp, average="macro"))
print("Recall: ",recall_score(y_true, y_pred_exp, average="macro"))
print("F1: ",f1_score(y_true, y_pred_exp,  average="macro"))
print("Accuracy: ",accuracy_score(y_true, y_pred_exp))

[[1139   73  118]
 [ 114 1112  142]
 [ 113   20 1234]]
              precision    recall  f1-score   support

        -1.0       0.83      0.86      0.84      1330
         0.0       0.92      0.81      0.86      1368
         1.0       0.83      0.90      0.86      1367

    accuracy                           0.86      4065
   macro avg       0.86      0.86      0.86      4065
weighted avg       0.86      0.86      0.86      4065

Precision:  0.8608711673022378
Recall:  0.8573210438108602
F1:  0.8573172000825698
Accuracy:  0.8573185731857319


#### ROC-AUC

In [ ]:
from sklearn.preprocessing import OneHotEncoder

In [ ]:
enc = OneHotEncoder(handle_unknown='ignore')
y_pred_roc = enc.fit_transform(np.array(y_pred_exp).reshape(-1,1))
Y_pred = y_pred_roc.toarray()

In [ ]:
y_true_r = enc.fit_transform(np.array(y_true).reshape(-1,1))
y_true_roc = y_true_r.toarray()

In [ ]:
 from sklearn.metrics import roc_auc_score
 roc_auc_score(y_true_roc,Y_pred, multi_class='ovr')

0.8930190868671114

### Top 10

In [ ]:
#example training
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
#[None, 'micro', 'macro', 'weighted'].
print(confusion_matrix(y_true,y_pred_exp))
print(classification_report(y_true,y_pred_exp))
print("Precision: ",precision_score(y_true, y_pred_exp, average="macro"))
print("Recall: ",recall_score(y_true, y_pred_exp, average="macro"))
print("F1: ",f1_score(y_true, y_pred_exp,  average="macro"))
print("Accuracy: ",accuracy_score(y_true, y_pred_exp))

[[1130   80  145]
 [ 125 1036  153]
 [ 112   40 1244]]
              precision    recall  f1-score   support

        -1.0       0.83      0.83      0.83      1355
         0.0       0.90      0.79      0.84      1314
         1.0       0.81      0.89      0.85      1396

    accuracy                           0.84      4065
   macro avg       0.84      0.84      0.84      4065
weighted avg       0.84      0.84      0.84      4065

Precision:  0.8431886370322944
Recall:  0.837832695292582
F1:  0.8386576123454162
Accuracy:  0.8388683886838868


# Pengujian 5

### Top 1

In [ ]:
#example training
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
#[None, 'micro', 'macro', 'weighted'].
print(confusion_matrix(y_true,y_pred_exp))
print(classification_report(y_true,y_pred_exp))
print("Precision: ",precision_score(y_true, y_pred_exp, average="macro"))
print("Recall: ",recall_score(y_true, y_pred_exp, average="macro"))
print("F1: ",f1_score(y_true, y_pred_exp,  average="macro"))
print("Accuracy: ",accuracy_score(y_true, y_pred_exp))

[[1139   73  118]
 [ 114 1112  142]
 [ 113   20 1234]]
              precision    recall  f1-score   support

        -1.0       0.83      0.86      0.84      1330
         0.0       0.92      0.81      0.86      1368
         1.0       0.83      0.90      0.86      1367

    accuracy                           0.86      4065
   macro avg       0.86      0.86      0.86      4065
weighted avg       0.86      0.86      0.86      4065

Precision:  0.8608711673022378
Recall:  0.8573210438108602
F1:  0.8573172000825698
Accuracy:  0.8573185731857319


#### ROC-AUC

In [ ]:
from sklearn.preprocessing import OneHotEncoder

In [ ]:
enc = OneHotEncoder(handle_unknown='ignore')
y_pred_roc = enc.fit_transform(np.array(y_pred_exp).reshape(-1,1))
Y_pred = y_pred_roc.toarray()

In [ ]:
y_true_r = enc.fit_transform(np.array(y_true).reshape(-1,1))
y_true_roc = y_true_r.toarray()

In [ ]:
 from sklearn.metrics import roc_auc_score
 roc_auc_score(y_true_roc,Y_pred, multi_class='ovr')

0.8930190868671114

### Top 5

In [ ]:
#example training
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
#[None, 'micro', 'macro', 'weighted'].
print(confusion_matrix(y_true,y_pred_exp))
print(classification_report(y_true,y_pred_exp))
print("Precision: ",precision_score(y_true, y_pred_exp, average="macro"))
print("Recall: ",recall_score(y_true, y_pred_exp, average="macro"))
print("F1: ",f1_score(y_true, y_pred_exp,  average="macro"))
print("Accuracy: ",accuracy_score(y_true, y_pred_exp))

[[1139   73  118]
 [ 114 1112  142]
 [ 113   20 1234]]
              precision    recall  f1-score   support

        -1.0       0.83      0.86      0.84      1330
         0.0       0.92      0.81      0.86      1368
         1.0       0.83      0.90      0.86      1367

    accuracy                           0.86      4065
   macro avg       0.86      0.86      0.86      4065
weighted avg       0.86      0.86      0.86      4065

Precision:  0.8608711673022378
Recall:  0.8573210438108602
F1:  0.8573172000825698
Accuracy:  0.8573185731857319


#### ROC-AUC

In [ ]:
from sklearn.preprocessing import OneHotEncoder

In [ ]:
enc = OneHotEncoder(handle_unknown='ignore')
y_pred_roc = enc.fit_transform(np.array(y_pred_exp).reshape(-1,1))
Y_pred = y_pred_roc.toarray()

In [ ]:
y_true_r = enc.fit_transform(np.array(y_true).reshape(-1,1))
y_true_roc = y_true_r.toarray()

In [ ]:
 from sklearn.metrics import roc_auc_score
 roc_auc_score(y_true_roc,Y_pred, multi_class='ovr')

0.8930190868671114

### Top 10

In [ ]:
#example training
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
#[None, 'micro', 'macro', 'weighted'].
print(confusion_matrix(y_true,y_pred_exp))
print(classification_report(y_true,y_pred_exp))
print("Precision: ",precision_score(y_true, y_pred_exp, average="macro"))
print("Recall: ",recall_score(y_true, y_pred_exp, average="macro"))
print("F1: ",f1_score(y_true, y_pred_exp,  average="macro"))
print("Accuracy: ",accuracy_score(y_true, y_pred_exp))

[[1130   80  145]
 [ 125 1036  153]
 [ 112   40 1244]]
              precision    recall  f1-score   support

        -1.0       0.83      0.83      0.83      1355
         0.0       0.90      0.79      0.84      1314
         1.0       0.81      0.89      0.85      1396

    accuracy                           0.84      4065
   macro avg       0.84      0.84      0.84      4065
weighted avg       0.84      0.84      0.84      4065

Precision:  0.8431886370322944
Recall:  0.837832695292582
F1:  0.8386576123454162
Accuracy:  0.8388683886838868


In [ ]:
#### ROC-AUC

from sklearn.preprocessing import OneHotEncoder

enc = OneHotEncoder(handle_unknown='ignore')
y_pred_roc = enc.fit_transform(np.array(y_pred_exp).reshape(-1,1))
Y_pred = y_pred_roc.toarray()

y_true_r = enc.fit_transform(np.array(y_true).reshape(-1,1))
y_true_roc = y_true_r.toarray()

from sklearn.metrics import roc_auc_score
roc_auc_score(y_true_roc,Y_pred, multi_class='ovr')

0.8784619009115726

### Top 20

In [ ]:
#example training
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
#[None, 'micro', 'macro', 'weighted'].
print(confusion_matrix(y_true,y_pred_exp))
print(classification_report(y_true,y_pred_exp))
print("Precision: ",precision_score(y_true, y_pred_exp, average="macro"))
print("Recall: ",recall_score(y_true, y_pred_exp, average="macro"))
print("F1: ",f1_score(y_true, y_pred_exp,  average="macro"))
print("Accuracy: ",accuracy_score(y_true, y_pred_exp))

[[1139   71  145]
 [ 130 1033  151]
 [ 132   46 1218]]
              precision    recall  f1-score   support

        -1.0       0.81      0.84      0.83      1355
         0.0       0.90      0.79      0.84      1314
         1.0       0.80      0.87      0.84      1396

    accuracy                           0.83      4065
   macro avg       0.84      0.83      0.83      4065
weighted avg       0.84      0.83      0.83      4065

Precision:  0.838581001317697
Recall:  0.8330774684805896
F1:  0.8340492200854971
Accuracy:  0.8339483394833949


In [ ]:
#### ROC-AUC

from sklearn.preprocessing import OneHotEncoder

enc = OneHotEncoder(handle_unknown='ignore')
y_pred_roc = enc.fit_transform(np.array(y_pred_exp).reshape(-1,1))
Y_pred = y_pred_roc.toarray()

y_true_r = enc.fit_transform(np.array(y_true).reshape(-1,1))
y_true_roc = y_true_r.toarray()

from sklearn.metrics import roc_auc_score
roc_auc_score(y_true_roc,Y_pred, multi_class='ovr')

0.8748534149411912